# SETUP

## Loading Library

In [64]:
#install.packages("caret")
#install.packages("tidyverse")
#install.packages("bartMachine")
#install.packages("doParallel")
library(caret)
library(tidyverse)
library(bartMachine)

In [43]:
#Include the parallel library. If the next line does not work, run install.packages(“parallel”) first
library(parallel)
library(doParallel)

# Use the detectCores() function to find the number of cores in system
n_cores <- detectCores()
print(n_cores)
cl <- makePSOCKcluster(n_cores)
#registerDoParallel(cl)
#stopCluster(cl)

[1] 16


## Loading Data

In [44]:
happiness.df <- read_csv(file="datasets/world-happiness/happiness.csv")
colnames(happiness.df) <- make.names(colnames(happiness.df) %>%  tolower)

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  X1 = col_double(),
  Country = col_character(),
  Region = col_character(),
  `Happiness Rank` = col_double(),
  `Happiness Score` = col_double(),
  `Economy (GDP per Capita)` = col_double(),
  Family = col_double(),
  `Health (Life Expectancy)` = col_double(),
  Freedom = col_double(),
  `Trust (Government Corruption)` = col_double(),
  Generosity = col_double(),
  `Dystopia Residual` = col_double(),
  year = col_double(),
  nyears = col_double()
)


We can get the train-test split using createDataPartition from caret:

In [45]:
happiness.df <- happiness.df %>% select_if(is_double) %>% select(-c(x1,happiness.rank))

In [46]:
set.seed(42)
trainIndex <- createDataPartition(happiness.df$`happiness.score`, p = 0.9, list = FALSE)
happy.train <- happiness.df[ trainIndex,]
happy.test  <- happiness.df[-trainIndex,]

## bartMachine

In [61]:
registerDoParallel(cl)

In [48]:
set_bart_machine_num_cores(n_cores)

bartMachine now using 16 cores.


Training: Run in 1hr 16 minutes on 4 cores:

In [49]:
t.start <- Sys.time()
happy.bm <- train(happiness.score ~ ., data = happy.train, 
                 method = "bartMachine", 
                 verbose = TRUE,
                 serialize = TRUE)
t.final <- Sys.time() - t.start

bartMachine initializing with 50 trees...
bartMachine vars checked...
bartMachine java init...
bartMachine factors created...
bartMachine before preprocess...
bartMachine after preprocess... 10 total features...
bartMachine sigsq estimated...
bartMachine training data finalized...
Now building bartMachine for regression ...
evaluating in sample data...done
serializing in order to be saved for future R sessions...done


In [50]:
print(t.final)

Time difference of 15.89687 mins


In [51]:
summary(happy.bm)

bartMachine v1.2.3 for regression

training data n = 426 and p = 9 
built in 2.3 secs on 16 cores, 50 trees, 250 burn-in and 1000 post. samples

sigsq est for y beforehand: 0 
avg sigsq estimate after burn-in: 0.00606 

in-sample statistics:
 L1 = 8.81 
 L2 = 0.34 
 rmse = 0.03 
 Pseudo-Rsq = 0.9994
p-val for shapiro-wilk test of normality of residuals: 0 
p-val for zero-mean noise: 0.98289 



In [52]:
 backup <-happy.bm

In [53]:
saveRDS(happy.bm,file = "model_files/20191118_bart_model.RDS")

In [54]:
happy.bm.read <- readRDS(file = "model_files/20191118_bart_model.RDS")

In [35]:
load("model_files/20191118_bart_model_save.RDS")

In [55]:
summary(happy.bm.read)

bartMachine v1.2.3 for regression

training data n = 426 and p = 9 
built in 2.3 secs on 16 cores, 50 trees, 250 burn-in and 1000 post. samples

sigsq est for y beforehand: 0 
avg sigsq estimate after burn-in: 0.00606 

in-sample statistics:
 L1 = 8.81 
 L2 = 0.34 
 rmse = 0.03 
 Pseudo-Rsq = 0.9994
p-val for shapiro-wilk test of normality of residuals: 0 
p-val for zero-mean noise: 0.98289 



In [56]:
predict(happy.bm.read,newdata = happy.test)

[1] 6.841406 6.662217 6.569922 6.497140 6.411975 6.283633 5.967950 5.959275
 [9] 5.820122 5.116915 4.892693 4.923138 4.546675 7.019797 6.715974 6.627295
[17] 6.272239 6.091003 6.001024 5.826738 5.533017 5.403893 5.189264 5.222956
[25] 4.381363 4.353948 4.204475 4.189179 3.961982 3.978235 3.835116 3.241017
[33] 6.964367 5.850758 5.823991 5.594764 5.280885 4.789813 4.722371 4.605835
[41] 4.551470 4.307190 4.281406 3.674384

In [63]:
stopCluster(cl)

ERROR: Error in summary.connection(connection): invalid connection


In [62]:
varImp(happy.bm.read)

.....


bartMachine variable importance

                              Overall
dystopia.residual             100.000
economy..gdp.per.capita.       61.131
family                         53.821
health..life.expectancy.       43.526
freedom                        35.008
generosity                     25.862
trust..government.corruption.  23.120
nyears                          5.119
year                            0.000